# PLS and Modelization

Author: Nhan TV

## Content<br>
<strong>1 Data</strong><br>
<strong>2  Method</strong><br>
2.1 Partial Least Squares <br>
2.2 Machine Learning Models<br>
2.3 Artificial Neural Network<br>
<strong>3 Accuracy Evaluate Methods</strong>


In [0]:
# Import all the libraries that we shall be using
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from sklearn import datasets
from scipy import stats
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score, mean_absolute_error, mean_squared_error
from sklearn.cluster import KMeans
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm
import collections


import xgboost as xgb

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [0]:
#a = []
#while(1):
    #a.append('1**100')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## 1 Data 

Load continue variables

In [0]:
chunksize = 100
X_ct = pd.DataFrame()
for chunk in pd.read_csv("/content/drive/My Drive/BI_rice/Xcont.csv", chunksize=chunksize):
    X_ct = pd.concat([X_ct, chunk])

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

Original continuous variables

In [0]:
X_ct.head()

,IID,IID.1,PAT,MAT,SEX,PHENOTYPE,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,16737_T,17263_C,21012_A,21546_A,22059_A,25777_A,27188_A,28781_T,31071_A,31478_C,31843_A,34721_T,34729_C,35239_A,35405_A,37676_A,41342_G,41749_C,42561_T,42752_T,44681_T,46283_G,46332_A,46705_A,...,373159983_T,373160477_T,373162865_A,373163410_T,373164165_T,373165126_G,373165826_G,373168073_A,373168114_T,373175331_A,373178803_T,373185083_A,373186111_A,373186311_G,373188112_T,373191891_A,373192677_T,373193439_C,373194339_T,373196011_G,373203579_T,373203871_G,373205568_G,373208796_T,373208813_C,373212798_T,373216011_C,373220537_T,373222966_T,373223422_A,373224710_T,373225563_G,373225815_T,373225923_G,373239794_A,373240105_G,373240943_T,373241264_C,373243874_T,373244037_C
0,IRIS_313-10000,IRIS_313-10000,0,0,0,2.6,0,2,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,2,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,IRIS_313-10001,IRIS_313-10001,0,0,0,2.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,IRIS_313-10002,IRIS_313-10002,0,0,0,2.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,IRIS_313-10007,IRIS_313-10007,0,0,0,2.7,1,2,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,2,0,1,...,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,2
4,IRIS_313-10010,IRIS_313-10010,0,0,0,2.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:

X_ct.columns

Index(['IID', 'IID.1', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', '1178_T', '1203_C',
       '1248_A', '1282_A',
       ...
       '373224710_T', '373225563_G', '373225815_T', '373225923_G',
       '373239794_A', '373240105_G', '373240943_T', '373241264_C',
       '373243874_T', '373244037_C'],
      dtype='object', length=404394)

Drop some variables that don't relate to model.

In [0]:
X_cont = X_ct.drop(['IID.1', 'PAT', 'MAT', 'SEX', 'PHENOTYPE'], axis = 1)

In [0]:
X_cont.set_index("IID", inplace = True)

In [0]:
X_cont.head()

,1178_T,1203_C,1248_A,1282_A,1299_C,1306_A,1792_A,5752_T,6634_A,11334_C,16737_T,17263_C,21012_A,21546_A,22059_A,25777_A,27188_A,28781_T,31071_A,31478_C,31843_A,34721_T,34729_C,35239_A,35405_A,37676_A,41342_G,41749_C,42561_T,42752_T,44681_T,46283_G,46332_A,46705_A,48241_A,48402_G,48494_T,49413_T,49960_A,50138_A,...,373159983_T,373160477_T,373162865_A,373163410_T,373164165_T,373165126_G,373165826_G,373168073_A,373168114_T,373175331_A,373178803_T,373185083_A,373186111_A,373186311_G,373188112_T,373191891_A,373192677_T,373193439_C,373194339_T,373196011_G,373203579_T,373203871_G,373205568_G,373208796_T,373208813_C,373212798_T,373216011_C,373220537_T,373222966_T,373223422_A,373224710_T,373225563_G,373225815_T,373225923_G,373239794_A,373240105_G,373240943_T,373241264_C,373243874_T,373244037_C
IID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
IRIS_313-10000,0,2,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,2,0,0,0,0,2,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IRIS_313-10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IRIS_313-10002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
IRIS_313-10007,1,2,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,2,0,1,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,2
IRIS_313-10010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Output variable

In [0]:
y_cont = X_ct["PHENOTYPE"]

In [0]:
y_cont.head()

0    2.6
1    2.6
2    2.5
3    2.7
4    2.2
Name: PHENOTYPE, dtype: float64

Split original data to training and testing set.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
...     X_cont, y_cont, test_size=0.2, random_state=0)

In [0]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1495, 404388) (1495,)
(374, 404388) (374,)


Normalize data

In [0]:
# Convert from integer to float
#X_train_fl = X_train.astype(float, 64)
#X_test_fl = X_test.astype(float, 64)

# Apply the same scaling to both datasets
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test) # note that we transform rather than fit_transform

## 2 Method

### 2.1 Partial Least Squares

Try to get number of principal component 

In [0]:
pls = PLSRegression(n_components=36)

In [0]:
pls.fit(X_train_scl, y_train)

PLSRegression(copy=True, max_iter=500, n_components=36, scale=True, tol=1e-06)

In [0]:
X_train_pls, y_train_pls = pls.transform(X_train_scl, y_train)

In [0]:
X_train_pls

array([[  -8.30892421,  -50.08927658,  113.30714443, ...,   14.42736957,
         -13.44531011,    1.62056755],
       [  27.19877434, -106.58536682,    7.14646538, ...,  -23.99217168,
          -0.51510407,    6.99418555],
       [  12.70117361,   63.15887372, -144.14478009, ...,  -16.25971945,
          -0.78372276,    6.20015297],
       ...,
       [ -16.3595994 ,  103.63799243,  -99.66435412, ...,  -15.35615239,
           9.38822766,  -20.79038312],
       [  13.28511615,  -98.19673186,   25.16608761, ...,   26.10484676,
         -14.82600128,   -1.96430146],
       [ -64.91525473,  -25.48597897,   17.18855618, ...,  -33.87552584,
           8.64272872,   13.67315334]])

Choosing principal components that explain data more than 99%.

In [0]:
score = pls.score(X_train_scl, y_train, sample_weight=None)

In [0]:
score

0.9950549072913343

In [0]:
X_test_pls, y_test_pls = pls.transform(X_test_scl, y_test)

Saving components after choosing.

In [0]:
from google.colab import files
np.savetxt('y_test.csv', y_test, delimiter=",") 
files.download('y_test.csv')

### 2.2 Machine Learning Models

Loading training and testing set.

In [0]:
X_train_pls = pd.read_csv("/content/drive/My Drive/BI_rice/X_train_pls.csv", header=None)
X_test_pls = pd.read_csv("/content/drive/My Drive/BI_rice/X_test_pls.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BI_rice/y_train.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BI_rice/y_test.csv", header=None)

Transforming data to matrix.

In [5]:
X_train_pls = X_train_pls.as_matrix()
X_test_pls = X_test_pls.as_matrix()
y_train = y_train.as_matrix().ravel()
y_test = y_test.as_matrix().ravel()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


#### 2.2.1 General Linear Model

Finding distribution of output variable.

In [0]:
def get_variable_distribution(y_train):
   
    value = []
 
    # Using kstest to identify continous variable distribution
    
    norm_distr = stats.kstest (y_train, 
                            "norm", stats.norm.fit(y_train)
                            )
    expon_distr = stats.kstest (y_train,
                            "expon", stats.expon.fit(y_train)
                            )
    gamma_distr = stats.kstest (y_train, 
                            "gamma", stats.gamma.fit(y_train)
                            )
                            
    min_error = min (norm_distr[0], expon_distr[0], gamma_distr[0])
    
    if norm_distr[0] == min_error and norm_distr[1] < 0.05:
        value = ['norm', norm_distr[0], norm_distr[1]]
        
    elif expon_distr[0] == min_error and expon_distr[1] < 0.05:
        value = ['expon', expon_distr[0], expon_distr [1]]
        
    elif gamma_distr[0] == min_error and gamma_distr[1] < 0.05:
        value = ['gamma', gamma_distr [0], gamma_distr [1]]

    return(value)

In [61]:
get_variable_distribution(y_train)

['gamma', 0.056701122098003265, 0.00012805647334822139]

In [62]:
model = sm.GLM(y_train, X_train_pls, family = sm.families.Gamma())
model_result = model.fit()
y_pred = model_result.predict(X_test_pls)
print("MAE test score:", mean_absolute_error(y_test, y_pred))
print("RMSE test score:", np.sqrt(mean_squared_error(y_test, y_pred)))

/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/generalized_linear_model.py:273: DomainWarning: The inverse_power link function does not respect the domain of the Gamma family.
  DomainWarning)


MAE test score: 2.9001769937085347
RMSE test score: 4.257666524343015


In [63]:
print(model_result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 1495
Model:                            GLM   Df Residuals:                     1459
Model Family:                   Gamma   Df Model:                           35
Link Function:          inverse_power   Scale:                          726.13
Method:                          IRLS   Log-Likelihood:                -11246.
Date:                Sun, 05 Jan 2020   Deviance:                       49614.
Time:                        10:49:06   Pearson chi2:                 1.06e+06
No. Iterations:                    74                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0133      0.012      1.139      0.2

#### 2.2.2 Random Forest Regressor

In [24]:
# Hyperparameters search grid 
rf_param_grid = {'bootstrap': [False, True],
         'n_estimators': [60, 70, 80, 90, 100],
         'max_features': [0.6, 0.65, 0.7, 0.75, 0.8],
         'min_samples_leaf': [1],
         'min_samples_split': [2]
        }

# Instantiate random forest classifier
rf_estimator = RandomForestRegressor(random_state=None)

# Create the GridSearchCV object
rf_model = GridSearchCV(estimator=rf_estimator, param_grid=rf_param_grid, cv=3, scoring='r2')

# Fine-tune the hyperparameters
rf_model.fit(X_train_pls, y_train)
print("Best Parameters:\n", rf_model.best_params_)

# Get the best model
rf_model_best = rf_model.best_estimator_

# Make predictions using the optimised parameters
rf_pred = rf_model_best.predict(X_test_pls)
print("MAE test score:", mean_absolute_error(y_test, rf_pred))
print("RMSE test score:", np.sqrt(mean_squared_error(y_test, rf_pred)))
#print('Random Forest accuracy:', round(r2_score(y_test, rf_pred), 3))

Best Parameters:
 {'bootstrap': False, 'max_features': 0.6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MAE test score: 0.4321256684491978
RMSE test score: 0.5593262825706758


In [25]:
rf_pred[:10]

array([2.246, 2.793, 2.409, 2.52 , 2.421, 3.007, 3.071, 2.401, 2.675,
       2.424])

In [26]:
y_test[:10]

array([3. , 2.2, 2.8, 1.9, 3. , 3. , 2.6, 3.6, 2.9, 2.6])

#### 2.2.3 Support Vector Machine Regressor

In [28]:
# Parameter grid
svm_param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 10], "kernel": ["rbf", "poly", "sigmoid"]} 

# Create SVM grid search regressor
svm_grid = GridSearchCV(SVR(), svm_param_grid, cv=3)

# Train the regressor
svm_grid.fit(X_train_pls, y_train)

print("Best Parameters:\n", svm_grid.best_params_)

# Select best svc
best_svr = svm_grid.best_estimator_

# Make predictions using the optimised parameters
svm_pred = best_svr.predict(X_test_pls)
print("MAE test score:", mean_absolute_error(y_test, svm_pred))
print("RMSE test score:", np.sqrt(mean_squared_error(y_test, svm_pred)))
#print('SVM accuracy:', round(r2_score(y_test, svm_pred), 3))

Best Parameters:
 {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
MAE test score: 0.42490025248995594
RMSE test score: 0.5462627029876432


In [29]:
svm_pred[:10]

array([2.32017795, 2.75645486, 2.40601667, 2.60765468, 2.41851567,
       2.79466183, 3.10391527, 2.45643459, 2.47972371, 2.33971204])

In [30]:
y_test[:10]

array([3. , 2.2, 2.8, 1.9, 3. , 3. , 2.6, 3.6, 2.9, 2.6])

#### 2.2.4 Xgboost Regressor

In [31]:
# XGB with no Grid Search 
xgb2_model = xgb.XGBRegressor()
xgb2_model.fit(X_train_pls, y_train)
xgb2_pred = xgb2_model.predict(X_test_pls)
print("MAE test score:", mean_absolute_error(y_test, xgb2_pred))
print("RMSE test score:", np.sqrt(mean_squared_error(y_test, xgb2_pred)))
#print('Accuracy: ', round(r2_score(y_test, xgb2_pred), 3))

[10:28:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE test score: 0.42403995761259344
RMSE test score: 0.5511331345113331


### 2.3 Artificial neural network

In [0]:
# ANN with PLS
def baseline_model(X, y):
  # Create model architecture
  model = Sequential()
  model.add(Dense(19, activation='linear', kernel_initializer='uniform',input_shape=(36,)))
  model.add(Dense(10, activation='linear', kernel_initializer='uniform'))
  model.add(Dense(1, activation='linear', kernel_initializer='uniform'))

  # Compile model
  model.compile(optimizer='adam',
  loss='mean_squared_error', 
  metrics=['mse'])

  # Fit model
  history = model.fit(X, y, epochs=100, batch_size=2)

  # Make predictions
  ann_pred = model.predict(X_test_pls)
  print("MAE test score:", mean_absolute_error(y_test, ann_pred))
  print("RMSE test score:", np.sqrt(mean_squared_error(y_test, ann_pred)))
  #print('Neural Network accuracy: ', round(r2_score(y_test, ann_pred), 3))
  return(model)

In [59]:
baseline_model(X_train_pls, y_train)

Epoch 1/100
1495/1495 [==============================] - 2s 1ms/step - loss: 3.2463 - mean_squared_error: 3.2463
Epoch 2/100
1495/1495 [==============================] - 1s 510us/step - loss: 0.0087 - mean_squared_error: 0.0087
Epoch 3/100
1495/1495 [==============================] - 1s 517us/step - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 4/100
1495/1495 [==============================] - 1s 502us/step - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 5/100
1495/1495 [==============================] - 1s 506us/step - loss: 0.0088 - mean_squared_error: 0.0088
Epoch 6/100
1495/1495 [==============================] - 1s 507us/step - loss: 0.0084 - mean_squared_error: 0.0084
Epoch 7/100
1495/1495 [==============================] - 1s 500us/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 8/100
1495/1495 [==============================] - 1s 500us/step - loss: 0.0125 - mean_squared_error: 0.0125
Epoch 9/100
1495/1495 [==============================] - 1s 501us/step - loss: 0.0

## 3 Accuracy evaluate methods